# Reinforcement learning algorithms primer

## Environment model (grid world)

In [ ]:
import numpy as np

BOARD_ROWS = 4
BOARD_COLS = 4
WIN_STATEs = [(0, 0), (3, 3)]
LOSE_STATEs = []
START = (2, 0)
DETERMINISTIC = True


class Environment:
    def __init__(self, state=START):

        self.actions = ["up", "down", "left", "right"]
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.board[1, 1] = -1
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC

        self.strategy = np.zeros([BOARD_ROWS, BOARD_COLS, len(self.actions)]) + 0.25

        self.state_values = {}
        self.reset_state_values_to_unknown()

    def get_random_nonterminal_state(self):
        terminal_state = True
        state = None
        while terminal_state:
            state = (np.random.randint(BOARD_ROWS), np.random.randint(BOARD_COLS))
            terminal_state = self.isTerminalState(state)
        return state

    def zeros_state_values(self):
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.state_values[(i, j)] = 0

    def reset_state_values_to_unknown(self):
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.isTerminalState((i, j)):
                    self.state_values[(i, j)] = 0
                else:
                    self.state_values[(i, j)] = -100000

    def giveReward(self):
        if self.state in (WIN_STATEs):
            return 0
        elif self.state in (LOSE_STATEs):
            return 0
        else:
            return -1

    def getReward(self, state, action):
        if self.isTerminalState(state):
            return 0
        else:
            return -1

        # next = self.getNextState(state, action)
        # if next == WIN_STATE:
        #    return 0
        # elif next == LOSE_STATE:
        #    return 0
        # else:
        #    return -1

    def getNextState(self, from_state, action):

        if self.isTerminalState(from_state):
            return from_state

        if action == "up":
            nxtState = (from_state[0] - 1, from_state[1])
        elif action == "down":
            nxtState = (from_state[0] + 1, from_state[1])
        elif action == "left":
            nxtState = (from_state[0], from_state[1] - 1)
        else:
            nxtState = (from_state[0], from_state[1] + 1)
        # if next state legal
        if (nxtState[0] >= 0) and (nxtState[0] <= (BOARD_ROWS - 1)):
            if (nxtState[1] >= 0) and (nxtState[1] <= (BOARD_COLS - 1)):
                # if nxtState != (1, 1):
                return nxtState

        return from_state

    def isTerminalState(self, state):
        if (state in (WIN_STATEs)) or (state in (LOSE_STATEs)):
            return True
        return False

    def nxtPosition(self, action):
        """
        action: up, down, left, right
        -------------
        0 | 1 | 2| 3|
        1 |
        2 |
        return next position
        """
        if self.determine:
            if action == "up":
                nxtState = (self.state[0] - 1, self.state[1])
            elif action == "down":
                nxtState = (self.state[0] + 1, self.state[1])
            elif action == "left":
                nxtState = (self.state[0], self.state[1] - 1)
            else:
                nxtState = (self.state[0], self.state[1] + 1)

        else:
            # non-deterministic
            action = self._chooseActionProb(action)
            nxtState = self.nxtPosition(action)

        # if next state legal
        if (nxtState[0] >= 0) and (nxtState[0] <= (BOARD_ROWS - 1)):
            if (nxtState[1] >= 0) and (nxtState[1] <= (BOARD_COLS - 1)):
                # if nxtState != (1, 1):
                return nxtState

        return (self.state)

    def nextStateReward(self, current_state, action):
        """
        action: up, down, left, right
        -------------
        0 | 1 | 2| 3|
        1 |
        2 |
        return next position
        """

        reward = self.getReward(current_state, action)
        nxtState = self.getNextState(current_state, action)

        return [nxtState, reward]

    def _chooseActionProb(self, action):
        if action == "up":
            return np.random.choice(["up", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "down":
            return np.random.choice(["down", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "left":
            return np.random.choice(["left", "up", "down"], p=[0.8, 0.1, 0.1])
        if action == "right":
            return np.random.choice(["right", "up", "down"], p=[0.8, 0.1, 0.1])

    def showStateValues(self):
        print('---------STATE VALUES------------')
        for i in range(0, BOARD_ROWS):
            print('----------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if True:
                    # "{0:0.2f}".format(self.state_values[(i, j)])
                    out += "{0:0.2f}".format(self.state_values[(i, j)]).ljust(6) + ' | '
            print(out)
        print('----------------------------------')

    def showStrategy(self):
        print('---------STRATEGY------------')
        for i in range(0, BOARD_ROWS):
            print('----------------------------------------------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if True:
                    out += str(self.strategy[i, j]).ljust(6) + ' | '
            print(out)
        print('---------------------------------------------------------------------------')

    def random_policy(self, state):
        return [1.0 / len(self.actions) for action in self.actions]

    def upright_policy(self, state):
        return [0.49, 0.01, 0.01, 0.49]

    def apply_strategy(self, state):
        return self.strategy[state[0], state[1]]

    def DP(self, gamma=1):
        state_value_improvement = True
        k = 0

        while state_value_improvement:

            state_value_improvement = False
            new_state_values = {}

            for state, v_old in self.state_values.items():
                if self.isTerminalState(state) == True:
                    new_state_values[state] = self.state_values[state]
                if self.isTerminalState(state) == False:
                    v_new = v_old
                    for j in range(len(self.actions)):
                        a = self.actions[j]
                        next_state, reward = self.nextStateReward(state, a)
                        next_state_transition_value = reward + self.state_values[next_state]
                        if v_new < next_state_transition_value:
                            v_new = next_state_transition_value
                    new_state_values[state] = v_new
                    if abs(v_old - v_new) > 0:
                        state_value_improvement = True
            self.state_values = new_state_values
            k = k + 1

    def value_iteration(self, epsilon, gamma=1):
        '''

        :param policy:
        :param epsilon:
        :param gamma:
        :return:
        '''

        k = 0
        state_value_improvement = True
        while state_value_improvement:
            state_value_improvement = False
            k = k + 1

            for state, v_old in self.state_values.items():
                if not self.isTerminalState(state):
                    action_reward = 0
                    for j in range(len(self.actions)):
                        action = self.actions[j]
                        prob = self.strategy[state[0], state[1], j]
                        next_state, reward = self.nextStateReward(state, action)
                        action_reward = action_reward + prob * (gamma * self.state_values[next_state] + reward)

                    if abs(self.state_values[state] - action_reward) > epsilon:
                        state_value_improvement = True

                    self.state_values[state] = action_reward
        print("Iterations", k)

    def greedily_update_policy(self, epsilon, gamma=1):
        for state, v_old in self.state_values.items():
            self.strategy[state[0], state[1]] = np.zeros(4)

            if not self.isTerminalState(state):
                action_reward = np.zeros(4)
                for j in range(len(self.actions)):
                    action = self.actions[j]
                    next_state, reward = self.nextStateReward(state, action)
                    action_reward[j] = gamma * self.state_values[next_state] + reward

                best_action = np.argmax(action_reward)
                self.strategy[state[0], state[1], best_action] = 1.


## Model free agent for Q-Learning

In [ ]:
class Agent:

    def __init__(self, zero_initial_values=False):
        # self.states_visited_trace = []  # record position and action taken at the position
        # self.states_reward_trace = []  # record position and action taken at the position

        # self.full_states_log=[]
        # self.actions = ["up", "down", "left", "right"]
        self.environment = Environment()
        # self.isEnd = False
        self.lr = 0.01
        self.exp_rate = 0.3
        self.decay_gamma = 1.0

        # initial state reward
        self.state_values = {}
        self.state_visit_count = {}
        self.current_state = self.environment.get_random_nonterminal_state()

        self.moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        if zero_initial_values:
            self.Q = np.zeros((BOARD_ROWS, BOARD_COLS, len(self.moves)))
        else:
            self.Q = np.random.uniform(0, 0.5, size=(BOARD_ROWS, BOARD_COLS, len(self.moves)))

    def chooseActionAccordingToPolicy(self, policy):
        # choose action with most expected value
        probs = policy
        cumulative_probs = np.cumsum(probs)
        action_index = np.digitize(np.random.uniform(0, 1), cumulative_probs)
        return self.environment.actions[action_index]

    def reset(self):
        self.states_visited_trace = []
        self.states_reward_trace = []
        self.environment = Environment()
        self.current_state = self.environment.get_random_nonterminal_state()

    def generate_episode(self):
        episode = []
        self.reset()

        while self.environment.isTerminalState(self.current_state) == False:
            state_from = self.current_state
            action = self.chooseActionAccordingToPolicy(self.environment.random_policy(state_from))
            target_state, reward = self.environment.nextStateReward(state_from, action)
            episode.append((state_from, action, reward))
            self.current_state = target_state
        return episode

    def q_learning_choose_action(self, episode, number_of_episodes, state):
        epsilon = 1 - episode / number_of_episodes

        if np.random.uniform(0, 1) < epsilon:
            moves_to_choose = []
            for move_ind, move in enumerate(self.moves):
                is_good_row = state[0] + move[0] in range(BOARD_ROWS)
                is_good_col = state[1] + move[1] in range(BOARD_COLS)
                if is_good_row and is_good_col:
                    moves_to_choose.append(move_ind)
            action = np.random.choice(moves_to_choose)
        else:
            action = np.argmax(self.Q[state])

        return action

    def q_learning_execute_action(self, state, action):

        if self.environment.isTerminalState(state):
            raise ValueError("Terminal current state: " + str(state))

        new_state = (
            max(0, min(BOARD_ROWS - 1, state[0] + self.moves[action][0])),
            max(0, min(BOARD_COLS - 1, state[1] + self.moves[action][1]))
        )

        if new_state[0] not in range(BOARD_ROWS) and new_state[1] not in range(BOARD_COLS):
            raise ValueError("Bad new state: " + str(new_state))

        return new_state

    def q_learning(self, number_of_episodes: int = 100, lr: float = 0.1, gamma: float = 0.99):

        for episode in range(number_of_episodes):

            current_state = self.current_state

            while not self.environment.isTerminalState(current_state):
                action = self.q_learning_choose_action(episode, number_of_episodes, current_state)
                next_state = self.q_learning_execute_action(current_state, action)
                _, reward = self.environment.nextStateReward(next_state, action)

                # print(f"Episode: {episode}, State: {current_state}, Action: {self.moves[action]}, Next State: {next_state}, Reward: {reward}")

                # Równanie Bellmana
                # print(f"Q[current_state][action]: {self.Q[current_state][action]}, np.max(self.Q[next_state]): {np.max(self.Q[next_state])}, self.Q[current_state][action]: {self.Q[current_state][action]}")
                self.Q[current_state][action] += lr * (
                        reward
                        + gamma * np.max(self.Q[next_state])
                        - self.Q[current_state][action]
                )

                current_state = next_state

    def q_learning_show_values(self):
        print(f"STATE VALUES".center(117, '-'))
        for i in range(0, BOARD_ROWS):
            print("-" * 117)
            out = '| '
            for j in range(0, BOARD_COLS):
                values_set = self.Q[i][j]
                values_str = ', '.join(["{0:0.2f}".format(value).ljust(5) for value in values_set])
                out += "{}".format(values_str).ljust(6) + ' | '
            print(out)
        print("-" * 117)


## Q Learning

In [ ]:
def q_learning_zeroes():
    print(f"Q-Learning with initial values 0".center(117, '-'))
    agent = Agent(zero_initial_values=True)
    agent.q_learning()
    agent.q_learning_show_values()
    print("-" * 117)


def q_learning_random():
    print(f"Q-Learning with initial values random".center(117, '-'))
    agent = Agent(zero_initial_values=False)
    agent.q_learning()
    agent.q_learning_show_values()
    print("-" * 117)


if __name__ == '__main__':
    q_learning_zeroes()
    q_learning_random()


-------------------------------------------Q-Learning with initial values 0------------------------------------------
-----------------------------------------------------STATE VALUES----------------------------------------------------
---------------------------------------------------------------------------------------------------------------------
| 0.00 , 0.00 , 0.00 , 0.00  | -0.10, -0.20, 0.00 , 0.00  | -0.20, -0.21, -0.27, -0.20 | -0.30, -0.30, -0.30, -0.20 | 
---------------------------------------------------------------------------------------------------------------------
| 0.00 , -0.28, 0.00 , -0.62 | -0.47, -0.45, -0.41, -0.42 | -0.34, -0.63, -0.40, -0.41 | -0.37, -0.34, -0.35, -0.30 | 
---------------------------------------------------------------------------------------------------------------------
| -0.65, -0.52, -0.20, -0.25 | -0.63, -0.68, -0.56, -0.61 | -0.77, -0.75, -0.79, -0.75 | -0.34, 0.00 , -0.57, 0.00  | 
-----------------------------------------------------